# Exploração dos dados de candidatos

[Voltar ao Índice](00_indice.ipynb)

Vamos verificar as características dos dados dos candidatos, quando completos com todas as informações vindas de todas as fontes (e.g. Bens, eleitorado, quantidade de votos). Especificamente, queremos verificar por que alguns candidatos não possume votos registrados.

In [1]:
import src.analysis_functions as af

## Carregando os dados

In [35]:
# Carrega informações sobre o candidato, seus bens e sobre o número de eleitores na sua unidade eleitoral:
cand_df = af.load_cand_eleitorado_bens_votos('../dados/limpos/consulta_cand/consulta_cand_2022_BRASIL_pos.csv',
                                             '../dados/limpos/perfil_eleitorado/perfil_eleitorado_2022.csv',
                                             '../dados/limpos/bens_candidatos/bem_candidato_2022_BRASIL.csv',
                                             '../dados/brutos/tse/votacao_candidato_munzona_2022/votacao_candidato_munzona_2022_BRASIL.csv',
                                             cand_sel_cols=['SQ_CANDIDATO', 'NM_CANDIDATO', 'NM_URNA_CANDIDATO', 'NR_CPF_CANDIDATO', 'NR_TITULO_ELEITORAL_CANDIDATO', 'SG_PARTIDO', 'SG_UF', 
                                                            'SG_UE', 'NM_UE', 'DS_CARGO', 'NM_SOCIAL_CANDIDATO', 'NR_IDADE_DATA_POSSE', 'DS_GENERO', 'DS_GRAU_INSTRUCAO', 'DS_COR_RACA',
                                                            'DS_SITUACAO_CANDIDATURA', 'DS_DETALHE_SITUACAO_CAND'])

## Candidatos sem votos

### Suplentes e vices recebem votos?

In [27]:
cargos_vice = ['1º SUPLENTE', '2º SUPLENTE', 'VICE-GOVERNADOR', 'VICE-PRESIDENTE']
vice_df = cand_df.loc[cand_df['DS_CARGO'].isin(cargos_vice)]

In [28]:
vice_df['QT_VOTOS_NOMINAIS'].value_counts(dropna=False)

NaN    805
Name: QT_VOTOS_NOMINAIS, dtype: int64

**Resultado:** Se temos apenas ausência de contabilização (NaN) para os suplentes e vices (veja acima), então eles não recebem votos

### E os demais cargos que não tem votos?

In [29]:
no_votes_df = cand_df.loc[cand_df['QT_VOTOS_NOMINAIS'].isnull() & ~cand_df['DS_CARGO'].isin(cargos_vice)]

In [30]:
no_votes_df['DS_DETALHE_SITUACAO_CAND'].value_counts()

INDEFERIDO                1048
RENÚNCIA                  1014
PEDIDO NÃO CONHECIDO        21
INDEFERIDO COM RECURSO      14
CANCELADO                   10
FALECIMENTO                  5
DEFERIDO                     2
Name: DS_DETALHE_SITUACAO_CAND, dtype: int64

In [31]:
no_votes_df['DS_SITUACAO_CANDIDATURA'].value_counts()

INAPTO    2098
APTO        16
Name: DS_SITUACAO_CANDIDATURA, dtype: int64

**Resultado:** Vemos que a maioria dos casos de candidatos sem votos são de pessoas vetadas.

Vamos investigar os casos de candidatos aptos mas que não receberam votos

In [32]:
aptos_no_votes_df = no_votes_df.loc[no_votes_df['DS_SITUACAO_CANDIDATURA'] == 'APTO']